In [1]:
import argparse, os, sys, shutil, glob, math
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
import subprocess

In [ ]:
# out_orig, out_masks, staging_input, staging_output = prepare_dirs(final_root)

# # read CSV
# df = pd.read_csv(csv_path)

# # sanity check columns
# for need in csv_cols.values():
#     if need not in df.columns:
#         raise SystemExit(f"CSV missing column {need}")

# saved_crops = 0
# total_rows  = 0

# # process per context to read each parquet once
# for ctx, g in df.groupby(csv_cols["ctx"]):
#     print(f"[context] {ctx}  rows {len(g)}")
#     cam_pq = pq_camera_image(training_root, ctx)
#     need_cols = [ts_col_camimg, cam_col_camimg, image_bytes_col]
#     if not all(c in cam_pq.columns for c in need_cols):
#         raise SystemExit(f"camera_image parquet missing required cols for {ctx}")
#     # build lookup by (timestamp, camera)
#     lookup = {(int(r[ts_col_camimg]), int(r[cam_col_camimg])): r for _, r in cam_pq.iterrows()}

#     for i, row in g.iterrows():
#         total_rows += 1
#         ts  = int(row[csv_cols["ts"]])
#         cam = int(row[csv_cols["cam"]])
#         key = (ts, cam)
#         r = lookup.get(key, None)
#         if r is None:
#             print(f"[warn] missing camera image for ts {ts} cam {cam}")
#             continue

#         img = decode_image_bytes(r, image_bytes_col)
#         if img is None:
#             print(f"[warn] decode fail for ts {ts} cam {cam}")
#             continue

#         cx = row[csv_cols["cx"]]; cy = row[csv_cols["cy"]]
#         w  = row[csv_cols["w"]] ;  h  = row[csv_cols["h"]]
#         crop = crop_from_center(img, cx, cy, w, h, expand=expand_factor)
#         if crop is None:
#             print(f"[warn] invalid crop for ts {ts} cam {cam}")
#             continue

#         stem = f"{ctx}_ts{ts}_cam{cam}_idx{i}"
#         cv2.imwrite(str(staging_input / f"{stem}.png"), crop)
#         cv2.imwrite(str(out_orig / f"{stem}.png"), crop)
#         saved_crops += 1

# print(f"[info] saved {saved_crops} crops out of {total_rows} rows")

# if saved_crops == 0:
#     raise SystemExit("no crops saved, nothing to run")


[context] 10023947602400723454_1120_000_1140_000  rows 571
[context] 1005081002024129653_5313_150_5333_150  rows 39
[context] 10061305430875486848_1080_000_1100_000  rows 9
[context] 10072140764565668044_4060_000_4080_000  rows 68
[context] 10075870402459732738_1060_000_1080_000  rows 24
[context] 10082223140073588526_6140_000_6160_000  rows 77
[context] 10094743350625019937_3420_000_3440_000  rows 4
[context] 10096619443888687526_2820_000_2840_000  rows 28
[context] 10206293520369375008_2796_800_2816_800  rows 13
[context] 10212406498497081993_5300_000_5320_000  rows 120
[context] 1022527355599519580_4866_960_4886_960  rows 690
[context] 10226164909075980558_180_000_200_000  rows 598
[context] 10231929575853664160_1160_000_1180_000  rows 565
[context] 10235335145367115211_5420_000_5440_000  rows 35
[context] 10241508783381919015_2889_360_2909_360  rows 229
[context] 10275144660749673822_5755_561_5775_561  rows 15
[context] 10391312872392849784_4099_400_4119_400  rows 58
[context] 1044

In [ ]:
# # run CDGNet once over the staging input
# run_cdgnet(
#     cdg_repo=cdg_repo,
#     weights=weights,
#     input_dir='C:\Waymo\crops_gait\_staging\input',
#     output_dir='C:\Waymo\crops_gait\_staging\output',
#     gpu="0",
#     batch_size=batch_size,
#     input_size=input_size,
#     num_classes=12,
#     vis="yes",
# )

# # # collect masks
# # mask_candidates = []
# # for patt in [
# #     str(staging_output / "parsing_results" / "*.png"),
# #     str(staging_output / "*.jpg"),
# #     str(staging_output / "parsing_results" / "*.jpg"),
# #     str(staging_output / "*.png"),
# # ]:
# #     mask_candidates.extend(glob.glob(patt))

# # if not mask_candidates:
# #     raise SystemExit(f"No masks found under {staging_output}. Adjust mask glob patterns.")

# # produced = {Path(p).stem.lower(): Path(p) for p in mask_candidates}


[run] "c:\Users\tient\miniconda3\envs\waymo\python.exe" "C:\Waymo\CDGNet-Parsing\inference.py" --data-dir "C:\Waymo\crops_gait\_staging\input" --gpu "0" --batch-size 4 --input-size "256,256" --restore-from "C:\Waymo\CDGNet-Parsing\model_best.pth" --dataset "val" --num-classes 12 --output-path "C:\Waymo\crops_gait\_staging\output" --vis "yes"


In [ ]:
orig_dir = Path(r"C:\Waymo\\crops_gait\\originals")
mask_dir = Path(r"C:\Waymo\\crops_gait\\masks")
stage_in_dir = Path(r"C:\Waymo\\crops_gait\\_staging\\input")
stage_out_dir = Path(r"C:\Waymo\\crops_gait\\_staging\\output\\Pred_parsing_results_vis")


exts = {".png", ".jpg", ".jpeg"}

orig_files = [p for p in orig_dir.glob("*") if p.suffix.lower() in exts]
mask_files = [p for p in mask_dir.glob("*") if p.suffix.lower() in exts]
stage_in_files = [p for p in stage_in_dir.glob("*") if p.suffix.lower() in exts]
stage_out_files = [p for p in stage_out_dir.glob("*") if p.suffix.lower() in exts]

print(f"originals count: {len(orig_files)}")
print(f"masks count:     {len(mask_files)}")
print(f"stage_in_files count: {len(stage_in_files)}")
print(f"stage_out_files count: {len(stage_out_files)}")
orig_stems = {p.stem.lower() for p in orig_files}
mask_stems = {p.stem.lower() for p in mask_files}

matched = orig_stems & mask_stems
missing_masks = orig_stems - mask_stems
missing_originals = mask_stems - orig_stems

print(f"matching pairs:  {len(matched)}")
if missing_masks:
    print(f"originals without masks: {len(missing_masks)}")
if missing_originals:
    print(f"masks without originals: {len(missing_originals)}")

originals count: 166262
masks count:     166262
stage_in_files count: 166262
stage_out_files count: 166262
matching pairs:  166262


In [2]:
from pathlib import Path
import pandas as pd, numpy as np

# uses your existing config vars:
# training_root, csv_path, final_root, expand_factor
csv_path = Path(r"C:\Waymo\\Gait3D\\people_gait_ready_waymo.csv")
training_root = Path(r"C:\Waymo\\training")
final_root = Path(r"C:\Waymo\\crops_gait")
CROPS_DIR = final_root / "originals"
OUT_SIDE  = None  # you didn’t resize to a fixed square in the code snippet; set to an int if you did later


csv_cols = { "ctx": "context", 
            "ts": "key.frame_timestamp_micros", 
            "cam": "key.camera_name", 
            "cx": "[CameraBoxComponent].box.center.x", 
            "cy": "[CameraBoxComponent].box.center.y", 
            "w": "[CameraBoxComponent].box.size.x", 
            "h": "[CameraBoxComponent].box.size.y", 
            "typ":"[CameraBoxComponent].type", 
}

expand_factor = 1.2


# CSV columns you declared
cols = {
    "ctx": csv_cols["ctx"],
    "ts":  csv_cols["ts"],
    "cam": csv_cols["cam"],
    "cx":  csv_cols["cx"],
    "cy":  csv_cols["cy"],
    "w":   csv_cols["w"],
    "h":   csv_cols["h"],
}

def roi_from_center(cx, cy, w, h, expand, H, W):
    # same clamping logic as your crop_from_center (no padding)
    w2 = max(1.0, float(w) * float(expand))
    h2 = max(1.0, float(h) * float(expand))
    x1 = int(round(float(cx) - w2 * 0.5))
    y1 = int(round(float(cy) - h2 * 0.5))
    x2 = int(round(float(cx) + w2 * 0.5))
    y2 = int(round(float(cy) + h2 * 0.5))
    x1c = max(0, min(W - 1, x1))
    y1c = max(0, min(H - 1, y1))
    x2c = max(0, min(W,     x2))
    y2c = max(0, min(H,     y2))
    if x2c <= x1c or y2c <= y1c:
        return None
    return int(x1c), int(y1c), int(x2c - x1c), int(y2c - y1c)

# load CSV
df = pd.read_csv(csv_path)
need = [cols[k] for k in ["ctx","ts","cam","cx","cy","w","h"]]
for c in need:
    if c not in df.columns:
        raise SystemExit(f"CSV missing {c}")

# index camera calib per context for H,W
def get_size_map(ctx):
    pq = pd.read_parquet(training_root / "camera_calibration" / f"{ctx}.parquet")
    # key.camera_name, width, height
    m = {}
    for _, r in pq.iterrows():
        cam = int(r["key.camera_name"])
        W = int(r["[CameraCalibrationComponent].width"])
        H = int(r["[CameraCalibrationComponent].height"])
        m[cam] = (W, H)
    return m

# we only keep rows whose crop file exists on disk
def expected_stem(ctx, ts, cam, idx):
    return f"{ctx}_ts{int(ts)}_cam{int(cam)}_idx{int(idx)}"

# pre-group for faster lookup
groups = {ctx: g for ctx, g in df.groupby(cols["ctx"])}

meta_rows = []
missing_calib = 0
missing_file  = 0
bad_roi       = 0

for ctx, g in groups.items():
    try:
        size_map = get_size_map(ctx)
    except Exception:
        missing_calib += len(g)
        continue

    for i, r in g.iterrows():
        ts  = int(r[cols["ts"]])
        cam = int(r[cols["cam"]])
        if cam not in size_map:
            missing_calib += 1
            continue

        W_full, H_full = size_map[cam]
        cx, cy = float(r[cols["cx"]]), float(r[cols["cy"]])
        w,  h  = float(r[cols["w"]]),  float(r[cols["h"]])

        roi = roi_from_center(cx, cy, w, h, expand_factor, H_full, W_full)
        if roi is None:
            bad_roi += 1
            continue
        x0,y0,wi,hi = roi

        stem = expected_stem(ctx, ts, cam, i)
        crop_path = CROPS_DIR / f"{stem}.png"
        if not crop_path.is_file():
            missing_file += 1
            continue

        meta_rows.append({
            "context": ctx,
            "ts": ts,
            "cam": cam,
            "x0": x0, "y0": y0, "w": wi, "h": hi,
            "Wc": wi if OUT_SIDE is None else OUT_SIDE,
            "Hc": hi if OUT_SIDE is None else OUT_SIDE,
            "stem": stem
        })

meta = pd.DataFrame(meta_rows)
out_meta = final_root / "crops_metadata.csv"
meta.to_csv(out_meta, index=False)
print(f"wrote {out_meta} rows={len(meta)}")
print(f"missing calib rows: {missing_calib}  missing crop files: {missing_file}  bad roi: {bad_roi}")


wrote C:\Waymo\crops_gait\crops_metadata.csv rows=166262
missing calib rows: 0  missing crop files: 0  bad roi: 0


In [3]:
import numpy as np

def adjust_intrinsics_for_crop(K_full, x0, y0, w, h, Wc, Hc):
    """
    K_full: 3x3 intrinsics of the original frame
    (x0,y0,w,h): crop rectangle in original pixels
    (Wc,Hc): crop image size you feed to the net (after any resize)
    """
    fx, fy = K_full[0,0], K_full[1,1]
    cx, cy = K_full[0,2], K_full[1,2]

    sx = float(Wc) / float(w)
    sy = float(Hc) / float(h)

    fx_p = fx * sx
    fy_p = fy * sy
    cx_p = (cx - x0) * sx
    cy_p = (cy - y0) * sy

    Kp = np.array([[fx_p, 0,    cx_p],
                   [0,    fy_p, cy_p],
                   [0,    0,    1.0]], dtype=np.float32)
    return Kp


In [4]:
from pathlib import Path
import pandas as pd, numpy as np, cv2

final_root = Path(r"C:\Waymo\\crops_gait")
meta_csv   = final_root / "crops_metadata.csv"
orig_dir   = final_root / "originals"   # not used here, just for reference
mask_dir   = final_root / "masks"       # crop-space masks from CDGNet
out_dir    = final_root / "full_masks_color"
out_dir.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(meta_csv)
print("rows:", len(df))

# build fast lookup for camera sizes
training_root = Path(r"C:\Waymo\\training")
sizes_cache = {}
def get_size(ctx, cam):
    key = (ctx, cam)
    if key in sizes_cache:
        return sizes_cache[key]
    pq = pd.read_parquet(training_root / "camera_calibration" / f"{ctx}.parquet")
    row = pq[pq["key.camera_name"] == cam].iloc[0]
    W = int(row["[CameraCalibrationComponent].width"])
    H = int(row["[CameraCalibrationComponent].height"])
    sizes_cache[key] = (W, H)
    return W, H

written = 0
skipped = 0

for _, r in df.iterrows():
    ctx, ts, cam = r["context"], int(r["ts"]), int(r["cam"])
    x0, y0, w, h = int(r["x0"]), int(r["y0"]), int(r["w"]), int(r["h"])
    stem = r["stem"]  # your file naming stem

    crop_mask_path = mask_dir / f"{stem}.png"
    if not crop_mask_path.is_file():
        skipped += 1
        continue

    crop_mask = cv2.imread(str(crop_mask_path), cv2.IMREAD_UNCHANGED)
    if crop_mask is None:
        skipped += 1
        continue

    W_full, H_full = get_size(ctx, cam)
    canvas = np.zeros((H_full, W_full, 3), dtype=np.uint8)

    # handle both colored masks (3ch) and id masks (single channel) without binarizing
    if crop_mask.ndim == 2:
        # single-channel ids -> keep ids by encoding them as colors with a fixed palette
        pal = np.zeros((256, 3), np.uint8)
        rng = np.random.RandomState(42)
        pal[1:] = rng.randint(0, 255, size=(255, 3), dtype=np.uint8)
        colored = pal[crop_mask.astype(np.uint8)]
        crop_color = colored
    else:
        # already BGR colors from CDGNet
        crop_color = crop_mask[:, :, :3]

    # if your saved crop size differs from w,h due to resize, align by resize back to (h,w)
    ch, cw = crop_color.shape[:2]
    if (cw, ch) != (w, h):
        crop_color = cv2.resize(crop_color, (w, h), interpolation=cv2.INTER_NEAREST)

    y1, y2 = y0, min(y0 + crop_color.shape[0], H_full)
    x1, x2 = x0, min(x0 + crop_color.shape[1], W_full)

    # paste where non-black to preserve body-part colors
    roi = canvas[y1:y2, x1:x2]
    cc  = crop_color[:(y2-y1), :(x2-x1)]
    mask_nonblack = (cc.sum(axis=2) > 0)
    roi[mask_nonblack] = cc[mask_nonblack]
    canvas[y1:y2, x1:x2] = roi

    out_name = f"{ctx}_ts{ts}_cam{cam}_fullpart.png"
    cv2.imwrite(str(out_dir / out_name), canvas)
    written += 1

print(f"full-frame colored masks written: {written}  skipped: {skipped}  -> {out_dir}")


rows: 166262
full-frame colored masks written: 166262  skipped: 0  -> C:\Waymo\crops_gait\full_masks_color


In [ ]:
# Streamed, low-memory painter for ONE frame:
# reads one full-frame colored mask (BGR), projects each LiDAR block to that camera,
# z-buffers in camera space, paints only those LiDAR pixels that land on non-black mask,
# concatenates per-lidar xyz + rgb, writes one NPZ. No giant idx_img or giant xyz_all kept at once.

from pathlib import Path
import numpy as np, pandas as pd, cv2, zlib, gc, json
import traceback

training_root = Path(r"C:\Waymo\\training")
full_mask_dir = Path(r"C:\Waymo\\crops_gait\\full_masks_color")
out_npz_dir   = Path(r"C:\Waymo\\human3d_npz")
out_npz_dir.mkdir(parents=True, exist_ok=True)

# persistent palette so the same mask colors map to the same part_id across frames
palette_json = out_npz_dir / "color_to_part_id.json"
if palette_json.is_file():
    with open(palette_json, "r") as f:
        color_to_id = {tuple(map(int,k.split(","))): int(v) for k,v in json.load(f).items()}
else:
    color_to_id = {}
next_id = 1 if not color_to_id else 1 + max(color_to_id.values())

def color_to_part_ids(colors_bgr):
    global next_id
    colors = [tuple(int(x) for x in c) for c in colors_bgr]
    ids = np.zeros(len(colors), dtype=np.int16)
    for i, c in enumerate(colors):
        if c == (0,0,0):
            ids[i] = 0
        else:
            if c not in color_to_id:
                color_to_id[c] = next_id
                next_id += 1
            ids[i] = color_to_id[c]
    return ids

def _decode(buf, H,W,C):
    arr = np.frombuffer(buf, np.float32)
    if arr.size != H*W*C:
        arr = np.frombuffer(zlib.decompress(buf), np.float32)
    return arr.reshape(H,W,C)

def paint_one_frame_streaming(ctx: str, ts: int, cam_name: int, use_return=1, top_only=False, verbose=True):
    # Camera calib for image size and vehicle->camera
    df_ccal = pd.read_parquet(training_root/"camera_calibration"/f"{ctx}.parquet")
    row_cam = df_ccal[df_ccal["key.camera_name"]==cam_name].iloc[0]
    W_img = int(row_cam["[CameraCalibrationComponent].width"])
    H_img = int(row_cam["[CameraCalibrationComponent].height"])
    T_vehicle_cam = np.array(row_cam["[CameraCalibrationComponent].extrinsic.transform"], np.float32).reshape(4,4)

    # Load colored full-frame mask
    mask_path = full_mask_dir / f"{ctx}_ts{ts}_cam{cam_name}_fullpart.png"
    mask_color = cv2.imread(str(mask_path), cv2.IMREAD_COLOR)
    if mask_color is None:
        if verbose: print("no mask:", mask_path.name)
        return False
    if mask_color.shape[:2] != (H_img, W_img):
        if verbose: print("mask size mismatch:", mask_color.shape, "vs", (H_img, W_img))
        return False

    # Load LiDAR, projection, and lidar calib rows for this ts
    df_li   = pd.read_parquet(training_root/"lidar"/f"{ctx}.parquet")
    df_prj  = pd.read_parquet(training_root/"lidar_camera_projection"/f"{ctx}.parquet")
    df_lcal = pd.read_parquet(training_root/"lidar_calibration"/f"{ctx}.parquet")

    tscol, ncol = "key.frame_timestamp_micros", "key.laser_name"
    ret_val   = "[LiDARComponent].range_image_return1.values" if use_return==1 else "[LiDARComponent].range_image_return2.values"
    ret_shape = "[LiDARComponent].range_image_return1.shape"  if use_return==1 else "[LiDARComponent].range_image_return2.shape"
    pj_val    = "[LiDARCameraProjectionComponent].range_image_return1.values" if use_return==1 else "[LiDARCameraProjectionComponent].range_image_return2.values"
    pj_shape  = "[LiDARCameraProjectionComponent].range_image_return1.shape"  if use_return==1 else "[LiDARCameraProjectionComponent].range_image_return2.shape"

    rows_li = df_li[df_li[tscol]==ts]
    if top_only:
        rows_li = rows_li[rows_li[ncol]==1]  # 1 is TOP

    # Per-frame accumulators, but keep them as lists and concat once
    xyz_blocks = []
    rgb_blocks = []
    part_blocks = []

    # We’ll also build a simple z-buffer per LiDAR block to resolve overlaps for THIS block
    # Then merge blocks by concatenation, since each block’s points are disjoint anyway.

    for _, r in rows_li.iterrows():
        try:
            laser = int(r[ncol])

            # Decode this lidar's range image
            H,W,C = map(int, r[ret_shape])
            ri = _decode(r[ret_val], H,W,C)
            rng = ri[...,0].astype(np.float32)

            # Beam inclinations
            crow = df_lcal[df_lcal[ncol]==laser].iloc[0]
            vals = crow.get("[LiDARCalibrationComponent].beam_inclination.values", None)
            if vals is not None:
                inc = np.asarray(vals, np.float32).ravel()
                if inc.size == H+1: inc = 0.5*(inc[:-1] + inc[1:])
                if inc.size != H:
                    inc = np.interp(np.linspace(0, inc.size-1, H, dtype=np.float32),
                                    np.arange(inc.size, dtype=np.float32), inc).astype(np.float32)
            else:
                inc = np.linspace(float(crow["[LiDARCalibrationComponent].beam_inclination.min"]),
                                  float(crow["[LiDARCalibrationComponent].beam_inclination.max"]),
                                  H, dtype=np.float32)

            az = np.linspace(-np.pi, np.pi, W, dtype=np.float32)
            incg = np.repeat(inc[:, None], W, axis=1)
            azg  = np.repeat(az[None, :], H, axis=0)

            x = rng * np.cos(incg) * np.cos(azg)
            y = rng * np.cos(incg) * np.sin(azg)
            z = rng * np.sin(incg)
            xyz_lidar = np.stack([x,y,z], axis=-1).reshape(-1,3)

            # Lidar -> vehicle
            T_vehicle_lidar = np.array(crow["[LiDARCalibrationComponent].extrinsic.transform"], np.float32).reshape(4,4)
            xyz_h = np.c_[xyz_lidar, np.ones((xyz_lidar.shape[0],1), np.float32)]
            xyz_vehicle = (T_vehicle_lidar @ xyz_h.T).T[:, :3]  # (N,3)

            # Get projection table for this lidar+return+timestamp
            sel = (df_prj[tscol]==ts) & (df_prj[ncol]==laser)
            if not sel.any():
                # append gray block, no coloring
                rgb_gray = np.full((xyz_vehicle.shape[0], 3), 180, np.uint8)
                part_ids = np.zeros(xyz_vehicle.shape[0], np.int16)
                xyz_blocks.append(xyz_vehicle.astype(np.float32))
                rgb_blocks.append(rgb_gray)
                part_blocks.append(part_ids)
                # free and continue
                del ri, rng, xyz_lidar, xyz_h
                gc.collect()
                continue

            prow = df_prj[sel].iloc[0]
            PH,PW,PC = map(int, prow[pj_shape])
            P = _decode(prow[pj_val], PH,PW,PC)   # (H,W,6) cam1,u1,v1, cam2,u2,v2

            cam1 = P[...,0].astype(np.int32); u1 = P[...,1]; v1 = P[...,2]
            cam2 = P[...,3].astype(np.int32); u2 = P[...,4]; v2 = P[...,5]
            m1 = cam1==cam_name
            m2 = cam2==cam_name

            # Linear indices into this lidar's range image
            rr1, cc1 = np.where(m1)
            rr2, cc2 = np.where(m2)
            lin = np.concatenate([rr1*PW + cc1, rr2*PW + cc2], axis=0)
            uu  = np.concatenate([u1[m1].astype(np.int32),      u2[m2].astype(np.int32)], axis=0)
            vv  = np.concatenate([v1[m1].astype(np.int32),      v2[m2].astype(np.int32)], axis=0)

            # z-buffer per lidar block
            # compute camera-space depth
            if lin.size:
                sel_xyz = xyz_vehicle[lin]
                cam_xyz = (T_vehicle_cam @ np.c_[sel_xyz, np.ones((sel_xyz.shape[0],1), np.float32)].T).T[:, :3]
                cam_z   = cam_xyz[:,2]
                ok = (uu>=0)&(uu<W_img)&(vv>=0)&(vv<H_img)&(cam_z>0.1)
            else:
                ok = np.zeros(0, dtype=bool)

            # start with default gray for this block
            rgb_bgr = np.full((xyz_vehicle.shape[0], 3), 180, np.uint8)
            part_ids = np.zeros(xyz_vehicle.shape[0], np.int16)

            if np.any(ok):
                uu_ok = uu[ok]; vv_ok = vv[ok]; lin_ok = lin[ok]; z_ok = cam_z[ok]

                # z-buffer within this block
                # sort by depth ascending, keep first occurrence per pixel
                order = np.argsort(z_ok)
                uu_ok = uu_ok[order]; vv_ok = vv_ok[order]; lin_ok = lin_ok[order]
                pix = vv_ok.astype(np.int64)*W_img + uu_ok.astype(np.int64)
                upix, first = np.unique(pix, return_index=True)
                uu_n = (upix % W_img).astype(np.int32)
                vv_n = (upix // W_img).astype(np.int32)
                lin_n = lin_ok[first]

                # sample colors from mask at nearest pixels
                sampled = mask_color[vv_n, uu_n]       # BGR
                nonblack = (sampled.sum(axis=1) > 0)

                if np.any(nonblack):
                    rgb_bgr[lin_n[nonblack]] = sampled[nonblack]
                    part_ids[lin_n[nonblack]] = color_to_part_ids(sampled[nonblack])

            # append this small block and free temporary arrays
            xyz_blocks.append(xyz_vehicle.astype(np.float32))
            rgb_blocks.append(rgb_bgr)
            part_blocks.append(part_ids)

            del ri, rng, xyz_lidar, xyz_h, P
            try: del sel_xyz, cam_xyz, cam_z
            except NameError: pass
            gc.collect()

        except Exception as e:
            print(f"[warn] lidar {laser} error:", e)
            traceback.print_exc(limit=1)
            gc.collect()
            continue

    if not xyz_blocks:
        if verbose: print("no lidar rows for", ctx, ts)
        return False

    # Concatenate per-lidar results for this frame and write NPZ
    xyz = np.vstack(xyz_blocks).astype(np.float32)
    rgb_bgr = np.vstack(rgb_blocks).astype(np.uint8)
    part = np.concatenate(part_blocks).astype(np.int16)

    # store as RGB in NPZ for training
    out_stem = f"{ctx}_ts{ts}_cam{cam_name}"
    out_npz = out_npz_dir / f"{out_stem}.npz"
    np.savez_compressed(out_npz,
                        xyz=xyz,
                        rgb=rgb_bgr[:, ::-1],   # BGR -> RGB in file
                        part_label=part,
                        ctx=ctx, ts=np.int64(ts), cam=np.int32(cam_name))

    if verbose:
        colored = int((rgb_bgr!=180).any(axis=1).sum())
        print(f"wrote {out_npz.name}: points {xyz.shape[0]}, colored {colored}, unique part ids {len(set(part.tolist()))- (1 if 0 in part else 0)}")

    # persist palette
    with open(palette_json, "w") as f:
        json.dump({",".join(map(str,k)): int(v) for k,v in color_to_id.items()}, f, indent=2)

    return True

# ----- test on ONE mask first -----
# Pick the first mask you have
masks = sorted(full_mask_dir.glob("*_fullpart.png"))
assert masks, "No masks found"
mpath = masks[0]
stem = mpath.stem.replace("_fullpart","")
ctx, rest = stem.split("_ts", 1)
ts_str, cam_str = rest.split("_cam")
ctx = ctx
ts  = int(ts_str)
cam = int(cam_str)

ok = paint_one_frame_streaming(ctx, ts, cam, use_return=1, top_only=False, verbose=True)
print("success:", ok)


wrote 10023947602400723454_1120_000_1140_000_ts1552440196662213_cam4.npz: points 649600, colored 236, unique part ids 6
success: True


In [1]:
from pathlib import Path
import json
import yaml

# inputs
palette_json = Path(r"C:\Waymo\\human3d_ply\\color_to_part_id.json")
out_dir      = Path(r"C:\Waymo\\human3d_out")
out_dir.mkdir(parents=True, exist_ok=True)

with open(palette_json, "r") as f:
    cmap = {tuple(map(int, k.split(","))): int(v) for k, v in json.load(f).items()}

# build id -> name table
ids = sorted(set(cmap.values()))
# reserve 0 as background or ignore
if 0 not in ids:
    ids = [0] + ids

part_db = {
    "num_labels": len(ids),
    "labels": {int(i): f"part_{int(i)}" for i in ids}
}
label_db = {
    "ignore_index": 0,
    "semantic_labels": ids
}

with open(out_dir / "part_database.yaml", "w") as f:
    yaml.safe_dump(part_db, f, sort_keys=False)
with open(out_dir / "label_database.yaml", "w") as f:
    yaml.safe_dump(label_db, f, sort_keys=False)

print("wrote", out_dir / "part_database.yaml")
print("wrote", out_dir / "label_database.yaml")

wrote C:\Waymo\human3d_out\part_database.yaml
wrote C:\Waymo\human3d_out\label_database.yaml
